# Regularization με Ridge & Lasso σε πραγματικό high‑dimensional dataset (Gas Sensors)

Σε αυτό το notebook θα δούμε πώς η **κανονικοποίηση (Ridge / Lasso)** βοηθά
ένα γραμμικό μοντέλο παλινδρόμησης όταν έχουμε:

- **λίγα δείγματα** και
- **πολλά χαρακτηριστικά (p ≫ n)**,

χρησιμοποιώντας το πραγματικό dataset
**Gas Sensor Array under Flow Modulation** από το UCI.



# Θεωρία και Διαισθητική Εξήγηση

Η **γραμμική παλινδρόμηση** προβλέπει μια συνεχή μεταβλητή στόχο $y$ από χαρακτηριστικά $x_1, \dots, x_p$,
με το γραμμικό μοντέλο:
$$
\hat{y} = \beta_0 + \beta_1 x_1 + \dots + \beta_p x_p.
$$

Στην κλασική **ελάχιστων τετραγώνων** (OLS) εκτιμούμε τα βάρη $\beta$ ελαχιστοποιώντας το μέσο τετραγωνικό σφάλμα (MSE):
$$
\min_{\boldsymbol{\beta}} \ \frac{1}{n}\sum_{i=1}^n \big(y_i - \hat{y}_i\big)^2.
$$

Όμως όταν υπάρχουν **πολλά χαρακτηριστικά**, **συσχέτιση (multicollinearity)** ή **θόρυβος**, η OLS μπορεί να:
- **υπερπροσαρμόζει (overfit)** στο train και να αποτυγχάνει στο test,
- έχει **ασταθείς συντελεστές** (πολύ μεγάλα ή με αλλαγές πρόσημου από μικρές μεταβολές στα δεδομένα).

Η **regularization (κανονικοποίηση)** προσθέτει ένα **πρόστιμο** στο μέγεθος των συντελεστών ώστε να ελέγχει την πολυπλοκότητα.

## Ridge (L2)
Η Ridge προσθέτει ποινή στο **τετράγωνο** των συντελεστών (L2):
$$
\min_{\boldsymbol{\beta}} \ \frac{1}{n}\sum_{i=1}^n \big(y_i - \hat{y}_i\big)^2 \;+\; \lambda \sum_{j=1}^p \beta_j^2.
$$

- Το $\lambda \ge 0$ ελέγχει τη **δύναμη** της ποινής: όσο μεγαλύτερο, τόσο περισσότερο **συρρικνώνονται** οι συντελεστές.
- Η Ridge **δεν μηδενίζει** χαρακτηριστικά, αλλά μειώνει τα βάρη τους και **σταθεροποιεί** τη λύση όταν υπάρχει multicollinearity.
- Συνήθως βελτιώνει τη **γενίκευση** (χαμηλότερο σφάλμα στο test) όταν το απλό μοντέλο υπερπροσαρμόζει.

## Lasso (L1)
Η Lasso προσθέτει ποινή στο **απόλυτο** μέγεθος των συντελεστών (L1):
$$
\min_{\boldsymbol{\beta}} \ \frac{1}{n}\sum_{i=1}^n \big(y_i - \hat{y}_i\big)^2 \;+\; \lambda \sum_{j=1}^p |\beta_j|.
$$

- Η Lasso έχει την ιδιότητα να **μηδενίζει** κάποιους συντελεστές για κατάλληλα $\lambda$ → λειτουργεί ως **feature selection**.
- Είναι χρήσιμη όταν πιστεύουμε ότι λίγα χαρακτηριστικά έχουν πραγματικά σημαντικό βάρος.

## Επιλογή $\lambda$ (υπερπαράμετρος)
Στην πράξη, επιλέγουμε το $\lambda$ με **cross‑validation** (π.χ. `RidgeCV`, `LassoCV`) ώστε να βρούμε την τιμή που δίνει το
καλύτερο σφάλμα στο validation set. Η σωστή regularization μειώνει το **variance** και βελτιώνει την **ικανότητα γενίκευσης**.

- Μικρό $\lambda$ → κοντά στην OLS → πιθανό overfitting.
- Μεγάλο $\lambda$ → έντονο shrinkage → underfitting.
- Ενδιάμεσο $\lambda$ → ισορροπία bias/variance και καλύτερη γενίκευση.


> 💡 **Σημείωση:** Στη βιβλιοθήκη scikit-learn η παράμετρος regularization συμβολίζεται ως `alpha`.  
> Δηλαδή, θεωρητικά $λ \equiv \texttt{alpha}$ — πρόκειται για το ίδιο μέγεθος που ελέγχει τη δύναμη της κανονικοποίησης.  
> Με άλλα λόγια, όσο μεγαλύτερο το `alpha` (ή $λ$), τόσο ισχυρότερη η ποινή και τόσο περισσότερο “συρρικνώνονται” οι συντελεστές.



## 1. Περιγραφή του dataset

Το dataset περιλαμβάνει μετρήσεις από **16 αισθητήρες αερίων** σε διάφορα
πρωτόκολλα ροής. Για κάθε πείραμα έχουμε:

- Μεταδεδομένα (π.χ. `exp`, `batch`, `gas`, `lab`)
- Συγκεντρώσεις δύο αερίων:
  - `ace_conc` – συγκέντρωση ακετόνης (στόχος που θα χρησιμοποιήσουμε)
  - `eth_conc` – συγκέντρωση αιθανόλης
- Χαρακτηριστικά (features) από την απόκριση των αισθητήρων:
  - μέγιστες/ελάχιστες τιμές ανά αισθητήρα,
  - χαρακτηριστικά χαμηλής/υψηλής συχνότητας κ.ά.

Στην πράξη, καταλήγουμε με **λίγα δείγματα (≈58)** και **πολλά features (≈400+)**.
Αυτό είναι κλασικό σενάριο όπου:

- η απλή γραμμική παλινδρόμηση (OLS) μπορεί να ταιριάξει σχεδόν τέλεια τα train δεδομένα,
- αλλά η γενίκευση στο test set μπορεί να είναι χειρότερη,
- ενώ τα μοντέλα Ridge/Lasso επιβάλλουν *πρόστιμο* στους συντελεστές και βελτιώνουν
  τη σταθερότητα.


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score


## 2. Φόρτωση δεδομένων

Περιμένουμε ότι έχετε κατεβάσει το αρχείο `features.csv` από το UCI και
το έχετε αποθηκεύσει ως:

```text
data/gas_features.csv
```

(βλ. οδηγίες στο `data/README.md`).

In [ ]:
data_path = Path("..") / "data" / "gas_features.csv"
if not data_path.exists():
    alt = Path("..") / "data" / "features.csv"
    if alt.exists():
        data_path = alt
    else:
        raise FileNotFoundError(
            f"Δεν βρέθηκε το {data_path}. "
            "Βεβαιωθείτε ότι έχετε κατεβάσει το features.csv από το UCI "
            "και το έχετε τοποθετήσει στον φάκελο data/."
        )

df = pd.read_csv(data_path)
df.head()


## 3. Επιλογή στόχου και χαρακτηριστικών

Ως μεταβλητή στόχο θα χρησιμοποιήσουμε τη συγκέντρωση ακετόνης `ace_conc`
και ως χαρακτηριστικά **όλες τις υπόλοιπες αριθμητικές στήλες**.

Έτσι προκύπτει ένα σενάριο με:

- $n$ (δείγματα) σχετικά μικρό,
- $p$ (χαρακτηριστικά) πολύ μεγαλύτερο από $n$.


In [ ]:
target_col = "ace_conc"

if target_col not in df.columns:
    raise ValueError(f"Δεν βρέθηκε η στήλη στόχος '{target_col}' στο DataFrame.")

y = df[target_col].astype("float64").values

X_num = df.drop(columns=[target_col]).select_dtypes(include=[np.number])
X = X_num.astype("float64").values

print("Σχήμα X:", X.shape)
print("Μέγεθος y:", y.shape)
print("Πρώτες numeric στήλες:", list(X_num.columns[:10]))


## 4. Διαχωρισμός σε Train / Test

Θα κρατήσουμε περίπου 80% για εκπαίδευση και 20% για test, με σταθερό `random_state`
για αναπαραγωγιμότητα.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape


## 5. Ορισμός μοντέλων: Linear, Ridge, Lasso

Θα δημιουργήσουμε 3 pipelines:

- **Linear Regression (OLS)**: baseline χωρίς κανονικοποίηση.
- **Ridge (L2)**: ποινή στο τετράγωνο των συντελεστών.
- **Lasso (L1)**: ποινή στο απόλυτο μέγεθος των συντελεστών (μπορεί να μηδενίσει κάποιους).

Κάθε pipeline θα περιλαμβάνει:

- `StandardScaler` για κλιμάκωση των χαρακτηριστικών.
- Το αντίστοιχο μοντέλο παλινδρόμησης.

Η υπερπαράμετρος κανονικοποίησης $\lambda$ στο scikit-learn ονομάζεται `alpha`
(ίδιο πράγμα, διαφορετικό σύμβολο).


In [ ]:
alphas_ridge = np.logspace(-3, 3, 13)
alphas_lasso = np.logspace(-3, 3, 13)

models = {
    "Linear": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),
    "Ridge": Pipeline([
        ("scaler", StandardScaler()),
        ("model", RidgeCV(alphas=alphas_ridge, cv=5))
    ]),
    "Lasso": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LassoCV(alphas=alphas_lasso, cv=5, max_iter=10000))
    ]),
}


## 6. Εκπαίδευση και αξιολόγηση

Θα εκπαιδεύσουμε κάθε μοντέλο και θα υπολογίσουμε:

- MSE και RMSE σε train & test,
- R² σε train & test,
- το **overfitting gap**: $\text{gap} = \text{RMSE}_{test} - \text{RMSE}_{train}$.

Μικρότερο |gap| σημαίνει πιο παρόμοια συμπεριφορά σε train και test
(δηλαδή λιγότερο overfitting ή underfitting).


In [ ]:
results = []

for name, pipe in models.items():
    print(f"\n=== {name} ===")
    pipe.fit(X_train, y_train)

    y_pred_train = pipe.predict(X_train)
    y_pred_test = pipe.predict(X_test)

    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
    rmse_train = np.sqrt(mse_train)
    rmse_test = np.sqrt(mse_test)
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    gap = rmse_test - rmse_train

    print(f"MSE  (train/test): {mse_train:.4f} / {mse_test:.4f}")
    print(f"RMSE (train/test): {rmse_train:.4f} / {rmse_test:.4f}")
    print(f"R²   (train/test): {r2_train:.4f} / {r2_test:.4f}")
    print(f"Gap (test - train): {gap:.4f}")

    alpha = None
    model = pipe.named_steps["model"]
    if hasattr(model, "alpha_"):
        alpha = float(model.alpha_)
        print(f"Επιλεγμένο alpha: {alpha:.6f}")

    results.append({
        "Model": name,
        "MSE_train": mse_train,
        "MSE_test": mse_test,
        "RMSE_train": rmse_train,
        "RMSE_test": rmse_test,
        "Gap": gap,
        "R2_train": r2_train,
        "R2_test": r2_test,
        "Alpha": alpha,
    })

results_df = pd.DataFrame(results)
results_df


## 7. Σύγκριση μοντέλων με bar chart

Θα συγκρίνουμε οπτικά:

- το RMSE στο test set,
- και το |gap| (πόσο διαφέρει το RMSE μεταξύ train και test).


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.bar(results_df["Model"], results_df["RMSE_test"])
plt.ylabel("RMSE (Test)")
plt.title("Σύγκριση RMSE στο test set")
plt.show()

plt.figure(figsize=(6, 4))
plt.bar(results_df["Model"], results_df["Gap"].abs())
plt.ylabel("|Gap| = |RMSE_test - RMSE_train|")
plt.title("Πόσο διαφορετικά συμπεριφέρονται σε train vs test")
plt.show()

plt.figure(figsize=(6, 4))
plt.bar(results_df["Model"], results_df["R2_test"])
plt.ylabel("R² (Test)")
plt.ylim(0.0, 1.05)
plt.title("Σύγκριση R² στο test set")
plt.show()


## 8. Ερμηνεία των αποτελεσμάτων

Σε αυτό το συγκεκριμένο dataset (λίγα δείγματα, πολλά features) βλέπουμε συνήθως:

- Η **γραμμική παλινδρόμηση (Linear)** μπορεί να έχει σχεδόν τέλειο fit στο train
  (πολύ χαμηλό RMSE, R²_train κοντά στο 1), αλλά μεγαλύτερο σφάλμα στο test.
- Η **Ridge** αφήνει λίγο μεγαλύτερο σφάλμα στο train, αλλά βελτιώνει το **RMSE_test**
  και μειώνει το overfitting gap. Οι συντελεστές της είναι "μαζεμένοι" γύρω από το 0.
- Η **Lasso** μπορεί να δώσει παρόμοιο ή και λίγο καλύτερο RMSE_test, με ακόμη
  πιο "απλό" μοντέλο (αρκετοί συντελεστές γίνονται ακριβώς 0).

Διαισθητικά:

- Με τόσα πολλά χαρακτηριστικά, η OLS μπορεί να "αποστηθίσει" το train set.
- Η κανονικοποίηση (Ridge/Lasso) προσθέτει μια ποινή στο μέγεθος των συντελεστών,
  ώστε το μοντέλο να είναι πιο **συνετό** και να γενικεύει καλύτερα σε νέα δεδομένα.


## Επιπλέον εποπτικά διαγράμματα

Για να κατανοήσουμε καλύτερα τη συμπεριφορά των μοντέλων:

- θα δούμε **πόσα χαρακτηριστικά χρησιμοποιεί ουσιαστικά** κάθε μοντέλο
  (μέσω του αριθμού των μη μηδενικών συντελεστών),
- και θα σχεδιάσουμε **διαγράμματα διασποράς (scatter plots)** με
  πραγματικές vs προβλεπόμενες τιμές στο test set, για να δούμε
  πόσο κοντά βρίσκονται οι προβλέψεις στη «ιδανική» ευθεία $y = x$.


In [ ]:
# Πόσους μη μηδενικούς συντελεστές έχει κάθε μοντέλο;
nnz_counts = {}

for name in results_df["Model"]:
    pipe = models[name]
    pipe.fit(X_train, y_train)
    model = pipe.named_steps["model"]
    if hasattr(model, "coef_"):
        coef = np.asarray(model.coef_).ravel()
        nnz_counts[name] = int(np.count_nonzero(coef))
    else:
        nnz_counts[name] = None

nnz_counts


In [ ]:
# Bar chart για τον αριθμό μη μηδενικών συντελεστών
plt.figure(figsize=(6, 4))
plt.bar(list(nnz_counts.keys()), list(nnz_counts.values()))
plt.ylabel("Μη μηδενικοί συντελεστές")
plt.title("Πόσο 'αραιό' είναι το μοντέλο (feature selection)")
plt.show()


In [ ]:
# Διαγράμματα πραγματικών vs προβλεπόμενων τιμών (test set)
for name in results_df["Model"]:
    pipe = models[name]
    pipe.fit(X_train, y_train)
    y_pred_test = pipe.predict(X_test)

    plt.figure(figsize=(5, 5))
    plt.scatter(y_test, y_pred_test)
    min_val = min(y_test.min(), y_pred_test.min())
    max_val = max(y_test.max(), y_pred_test.max())
    plt.plot([min_val, max_val], [min_val, max_val])
    plt.xlabel("Πραγματικές τιμές (y_test)")
    plt.ylabel("Προβλεπόμενες τιμές")
    plt.title(f"Real vs Predicted - {name}")
    plt.show()


## Σύνοψη: Γιατί η Lasso φαίνεται διαφορετική από τη Ridge;

Στην αρχή του notebook παρουσιάσαμε τη θεωρία της γραμμικής παλινδρόμησης και της κανονικοποίησης (Ridge / Lasso) και είδαμε ότι
η μόνη ουσιαστική διαφορά στους δύο τύπους regularization είναι στο **είδος της ποινής**:

- Ridge (L2): ποινή στο τετράγωνο των συντελεστών, τείνει να τους **συρρικνώνει** αλλά σπάνια τους μηδενίζει.
- Lasso (L1): ποινή στο απόλυτο μέγεθος των συντελεστών, μπορεί να κάνει πολλούς $\beta_j$ **ακριβώς μηδέν**.

Στο συγκεκριμένο high‑dimensional πρόβλημα (λίγα δείγματα, πολλά χαρακτηριστικά) είδαμε στα αποτελέσματα και στα γραφήματα ότι:

- Η απλή OLS/Linear Regression σχεδόν «αποστηθίζει» το train set (πολύ υψηλό $R^2$ στο train),
  αλλά έχει χειρότερο σφάλμα στο test.
- Η Ridge αποδέχεται λίγο μεγαλύτερο σφάλμα στο train, αλλά βελτιώνει το $R^2$ και το RMSE στο test
  και έχει μικρότερο overfitting gap.
- Η Lasso πετυχαίνει παρόμοιο ή και καλύτερο $R^2$ στο test **με λιγότερους μη μηδενικούς συντελεστές**,
  δηλαδή με ένα πιο απλό και αραιό μοντέλο (feature selection).

Τα bar charts για $R^2$, για το $|\text{gap}|$ και για τον αριθμό μη μηδενικών συντελεστών δείχνουν ξεκάθαρα ότι:

- η Ridge είναι πιο «συντηρητική»: κρατά όλα τα features αλλά με πιο μικρά βάρη,
- η Lasso είναι πιο «επιθετική»: πετάει αρκετά features (συντελεστές ακριβώς 0) και κρατά μόνο όσα συμβάλλουν περισσότερο.

Συνεπώς, η διαφορετική συμπεριφορά της Lasso δεν είναι τυχαία· είναι άμεση συνέπεια της L1 ποινής και
την καθιστά ιδιαίτερα χρήσιμη όταν:

- έχουμε πολλά χαρακτηριστικά και υποψιαζόμαστε ότι μόνο λίγα είναι πραγματικά σημαντικά,
- θέλουμε ένα μοντέλο πιο εύκολα ερμηνεύσιμο (λιγότερα ενεργά features),
- θέλουμε ταυτόχρονα regularization *και* ενσωματωμένο feature selection.
